<a href="https://colab.research.google.com/github/NandakrishnanR/Kaggle_projects/blob/master/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jsphyg/weather-dataset-rattle-package")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'weather-dataset-rattle-package' dataset.
Path to dataset files: /kaggle/input/weather-dataset-rattle-package


In [92]:
import pandas as pd
raw_df=pd.read_csv(f"/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")

In [93]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
import os
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [94]:
print(raw_df.isnull().sum())

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


In [95]:
#Given that 'RainTomorrow' is almost certainly your target variable for a prediction task, it's generally best to remove the rows where 'RainTomorrow' is null. Imputing a categorical target variable can lead to your model learning from artificially generated labels, making your model's evaluation and predictions unreliable. You have 3267 missing values for 'RainTomorrow' out of 145,460 entries, which is about 2.2%. Removing these rows is a reasonable approach to ensure data quality for your target variable.
raw_df.dropna(subset = ['RainTomorrow'], inplace = True)


## Preparing the Data for Training

steps to prepare the dataset for training:

1. Create a train/test/validation split
2. Identify input and target columns
3. Identify numeric and categorical columns
4. Impute (fill) missing numeric values
5. Scale numeric values to the $(0, 1)$ range
6. Encode categorical columns to one-hot vectors

In [96]:
year = pd.to_datetime(raw_df['Date']).dt.year
raw_df['year'] = year
from plotly import express as px
px.histogram(raw_df, x='year', color='RainToday', title='Rain Today by Year')

In [97]:
train_df=raw_df[raw_df['year']<2016]
val_df=raw_df[raw_df['year'] == 2016]
test_df=raw_df[raw_df['year'] > 2016]

In [98]:
input_col=list(raw_df.columns)
input_col.remove('RainTomorrow')
input_col.remove('Date')
input_col.remove('year')
target_col='RainTomorrow'
train_input=train_df[input_col].copy()
train_target=train_df[target_col].copy()
val_input=val_df[input_col].copy()
val_target=val_df[target_col].copy()
test_input=test_df[input_col].copy()
test_target=test_df[target_col].copy()


In [99]:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import pandas as pd



#  Numeric columns
num_cols = train_input.select_dtypes(include=['int64','float64']).columns.tolist()

imputer = SimpleImputer(strategy='mean')
imputer.fit(train_input[num_cols])

train_num = imputer.transform(train_input[num_cols])
val_num   = imputer.transform(val_input[num_cols])
test_num  = imputer.transform(test_input[num_cols])

scaler = MinMaxScaler()
scaler.fit(train_num)

train_num = scaler.transform(train_num)
val_num   = scaler.transform(val_num)
test_num  = scaler.transform(test_num)

train_num_df = pd.DataFrame(train_num, columns=num_cols, index=train_input.index)
val_num_df   = pd.DataFrame(val_num,   columns=num_cols, index=val_input.index)
test_num_df  = pd.DataFrame(test_num,  columns=num_cols, index=test_input.index)

# Categorical columns
cat_cols = train_input.select_dtypes(include=['object','string','category']).columns.tolist()

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe.fit(train_input[cat_cols])

enc_cols = ohe.get_feature_names_out().tolist()

train_cat_df = pd.DataFrame(ohe.transform(train_input[cat_cols]), columns=enc_cols, index=train_input.index)
val_cat_df   = pd.DataFrame(ohe.transform(val_input[cat_cols]),   columns=enc_cols, index=val_input.index)
test_cat_df  = pd.DataFrame(ohe.transform(test_input[cat_cols]),  columns=enc_cols, index=test_input.index)

# Combine (numeric scaled + categorical encoded)
X_train = pd.concat([train_num_df, train_cat_df], axis=1)
X_val   = pd.concat([val_num_df,   val_cat_df],   axis=1)
X_test  = pd.concat([test_num_df,  test_cat_df],  axis=1)

print("Train shape:", X_train.shape)
print("Val shape:",   X_val.shape)
print("Test shape:",  X_test.shape)

# training
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=8, random_state=42)
model.fit(X_train, train_target)



Train shape: (116219, 119)
Val shape: (17508, 119)
Test shape: (8466, 119)


DecisionTreeClassifier(max_depth=8, random_state=42)

In [107]:
print("classes:", model.classes_)
model.predict_proba(X_train)

classes: ['No' 'Yes']


array([[0.93426819, 0.06573181],
       [0.92406389, 0.07593611],
       [0.92406389, 0.07593611],
       ...,
       [0.92406389, 0.07593611],
       [0.8546169 , 0.1453831 ],
       [0.92406389, 0.07593611]])

In [104]:
train_prediction=model.predict(X_train)
training_accuracy_score=model.score(X_train,train_target)
print(training_accuracy_score)
val_prediction=model.predict(X_val)
val_accuracy_score=model.score(X_val,val_target)
print(val_accuracy_score)
test_prediction=model.predict(X_test)
test_accuracy_score=model.score(X_test,test_target)
print(test_accuracy_score)



0.852433767284179
0.826022389764679
0.848334514528703


In [108]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(train_target,train_prediction))


[[85729  4559]
 [12591 13340]]


In [120]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, classification_report, roc_auc_score
)

print("classes:", model.classes_)  # should be ['No' 'Yes']
pos = 'Yes'
pos_idx = np.where(model.classes_ == pos)[0][0]

def evaluate_split(X, y, name):
    y_prob_pos = model.predict_proba(X)[:, pos_idx]
    y_hat = (y_prob_pos >= 0.70).astype(int)               #  threshold
    y_true = (y == pos).astype(int)

    acc = accuracy_score(y_true, y_hat)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_hat, average='binary', zero_division=0
    )
    auc = roc_auc_score(y_true, y_prob_pos)
    print(f"\n{name}")
    print("Accuracy:", round(acc, 4), "Precision:", round(prec, 4),
          "Recall:", round(rec, 4), "F1:", round(f1, 4), "ROC-AUC:", round(auc, 4))


evaluate_split(X_train, train_target, "TRAIN")
evaluate_split(X_val,   val_target,   "VAL")
evaluate_split(X_test,  test_target,  "TEST")

classes: ['No' 'Yes']

TRAIN
Accuracy: 0.838 Precision: 0.8644 Recall: 0.3247 F1: 0.4721 ROC-AUC: 0.8667

VAL
Accuracy: 0.819 Precision: 0.8137 Recall: 0.3144 F1: 0.4535 ROC-AUC: 0.8321

TEST
Accuracy: 0.8369 Precision: 0.8173 Recall: 0.2791 F1: 0.4161 ROC-AUC: 0.8364
